<a href="https://colab.research.google.com/github/bas-ik/study_practise/blob/main/practice_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install nltk
!pip install pymorphy2
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import warnings
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [25]:
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000


np.random.seed(17)

### Data description



1. – id: уникальный номер сообщения в системе twitter;
2. – date: дата публикации сообщения (твита);
3. – user_name: имя пользователя, опубликовавшего сообщение;
4. – text: текст сообщения (твита);
5. – target: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
6. – trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
7. – twits_count: количество ретвитов;
8. – favorites_count: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
9. – user_twits_count: число всех сообщений пользователя в сети twitter;
10. – fol_count: количество фоловеров пользователя (тех людей, которые читают пользователя);
11. – friend_count: количество друзей пользователя (те люди, которых читает пользователь);
12. – list_count: количество листов-подписок в которые добавлен твиттер-пользователь.


In [26]:
cols = ['id', 'date', 'user_name', 'text', 'target', 'replay_count', 'twits_count','favorites_count', 'user_twits_count', 'fol_count', 'friend_count', 'list_count']

df_pos = pd.read_csv("/content/drive/MyDrive/dataset_for_practice_work/positive.csv", sep=';', header=None, names=cols, index_col='id')
df_neg = pd.read_csv("/content/drive/MyDrive/dataset_for_practice_work/negative.csv", sep=';', header=None, names=cols, index_col='id')

#preprocessing

In [27]:
# we need only text and target for classification

df_pos.drop(['date', 'user_name', 'replay_count', 'twits_count','favorites_count', 'user_twits_count', 'fol_count', 'friend_count', 'list_count'], axis=1, inplace=True)
df_neg.drop(['date', 'user_name', 'replay_count', 'twits_count','favorites_count', 'user_twits_count', 'fol_count', 'friend_count', 'list_count'], axis=1, inplace=True)

# negative target is usually indicated by 0
df_neg['target'] = 0

# concat positive and negative df and drop duplicates to not overlearn model  
df = df_pos.append(df_neg)
df = df.dropna().drop_duplicates()

In [28]:
df.head(-5)

,text,target
id,,
408906692374446080,"@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)",1
408906692693221377,"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D",1
408906695083954177,RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!,1
408906695356973056,"RT @digger2912: ""Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим"" :DD http://t.co/GqG6iuE2…",1
408906761416867842,"@irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D",1
...,...,...
425137498151735296,"@ivanenko14 и у меня также, только будильник еще и не выключался.. папу разбудила (",0
425137763466633216,Когда же я вернусь к тебе(( http://t.co/36qnCOGdXr,0
425137932283158528,"RT @qelasocadij: Скажите пожалуйста, как у человека может быть 1000 одноклассников? O_o",0


In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
ru_stopwords = stopwords.words("russian")
morph = MorphAnalyzer()


def clean_data(data):
  data = re.sub(patterns, ' ', data)
  tokens = []
  for token in data.split():
    # print(token)
    if token and token not in ru_stopwords:
      token = token.strip()
      token = morph.normal_forms(token)[0]
      tokens.append(token)
  
  return tokens

In [32]:
data = df['text'].apply(clean_data)

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


In [33]:
data['target'] = df['target']

In [34]:
d = {"text": data, "target": df['target']}
cleaned_df = pd.DataFrame(data=d)


In [35]:
cleaned_df = cleaned_df.dropna()
cleaned_df['target'] = cleaned_df['target'].astype('int')

In [36]:
cleaned_df['text'] = cleaned_df['text'].apply(lambda x: " ".join(map(str, x)))

In [37]:
# удалим строки с пропущенными значениями
cleaned_df.dropna(how='any', inplace=True);

# проверим на наличие нулевых строк
cleaned_df.isna().sum()

text      0
target    0
dtype: int64

In [38]:
# запишем обработанные данные, чтобы при каждом запуске колаба не ждать по 10 минут
cleaned_df.to_csv('cleaned_data.csv', index=False)

# read prepared data 

### этот блок нужен только для загрузки ранее обработанных данных

In [ ]:
cleaned_df = pd.read_csv('/content/drive/MyDrive/dataset_for_practice_work/cleaned_data.csv')
cleaned_df.head()

,text,target
0,школотый поверь самый общество профилировать предмет тип,1
1,да таки немного похожий но мальчик равно,1
2,ну идиотка испугаться,1
3,кто угол сидеть погибать голод ещё порция взять хотя жрать хотеть …,1
4,вот значит страшилка но блин посмотреть часть создаться ощущение автор курить,1


In [ ]:
cleaned_df['text'] = cleaned_df['text'].astype('List')
cleaned_df.dtypes

# vectorization and training

In [39]:
def tfidf_LogReg(X_train, X_test, y_train, y_test):
  vectorizer = TfidfVectorizer()

  tf_x_train = vectorizer.fit_transform(X_train)
  tf_x_test = vectorizer.transform(X_test)


  logreg = LogisticRegression(penalty='l2', solver='newton-cg', C=1)
  logreg.fit(tf_x_train, y_train)
  y_test_pred = logreg.predict(tf_x_test)
  report=classification_report(y_test, y_test_pred, output_dict=True)
  return report['macro avg']['f1-score']


In [40]:
def BOW_LogReg(X_train, X_test, y_train, y_test, n_gram=1):
  vectorizer = CountVectorizer(ngram_range=(n_gram, n_gram))

  tf_x_train = vectorizer.fit_transform(X_train)
  tf_x_test = vectorizer.transform(X_test)


  logreg = LogisticRegression(penalty='l2', solver='newton-cg', C=1)
  logreg.fit(tf_x_train, y_train)
  y_test_pred = logreg.predict(tf_x_test)
  report=classification_report(y_test, y_test_pred, output_dict=True)
  return report['macro avg']['f1-score']


In [41]:
X_train, X_test, y_train, y_test = train_test_split(cleaned_df['text'], cleaned_df['target'], test_size=0.3, random_state=17)

BOW_res_1 = BOW_LogReg(X_train, X_test, y_train, y_test, 1)
BOW_res_2 = BOW_LogReg(X_train, X_test, y_train, y_test, 2)
BOW_res_3 = BOW_LogReg(X_train, X_test, y_train, y_test, 3)
tfidf_res = tfidf_LogReg(X_train, X_test, y_train, y_test)

In [42]:
primary_result = (BOW_res_1, BOW_res_2, BOW_res_3, tfidf_res)

In [43]:
primary_result
print(f"BOW-1: {primary_result[0]}\nBOW-2: {primary_result[1]}\nBOW-3: {primary_result[2]}\nTF-IDF: {primary_result[3]}")

BOW-1: 0.7152191969366974
BOW-2: 0.6337810220904445
BOW-3: 0.44242587283308876
TF-IDF: 0.7175604833247469


| *vectorization type*  | *f1-score* |
| ------------- | ------------- |
| bag of words 1-gram  | **0.7152191969366974**  |
| bag of words 2-gram  | **0.6337810220904445**  |
| bag of words 1-gram  | **0.44242587283308876**  |
| tf-idf| **0.7175604833247469** |